In [63]:
import requests
import json
from fuzzywuzzy import process
import pyaudio
import wave
import pyttsx3
import datetime
import spacy
import re
from pyowm import OWM

def record_audio(RECORD_SECONDS, WAVE_OUTPUT_FILENAME):
    # --------- SETTING PARAMS FOR OUR AUDIO FILE ------------#
    FORMAT = pyaudio.paInt16  # format of wave
    CHANNELS = 2  # no. of audio channels
    RATE = 44100  # frame rate
    CHUNK = 1024  # frames per audio sample
    # --------------------------------------------------------#

    # creating PyAudio object
    audio = pyaudio.PyAudio()

    # open a new stream for microphone
    # It creates a PortAudio Stream Wrapper class object
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    # ----------------- start of recording -------------------#
    print("Listening...")

    # list to save all audio frames
    frames = []

    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        # read audio stream from microphone
        data = stream.read(CHUNK)
        # append audio data to frames list
        frames.append(data)

    # ------------------ end of recording --------------------#
    print("Finished recording.")

    stream.stop_stream()  # stop the stream object
    stream.close()  # close the stream object
    audio.terminate()  # terminate PortAudio

    # ------------------ saving audio ------------------------#

    # create wave file object
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')

    # settings for wave file object
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))

    # closing the wave file object
    waveFile.close()


def read_audio(WAVE_FILENAME):
    # function to read audio(wav) file
    with open(WAVE_FILENAME, 'rb') as f:
        audio = f.read()
    return audio

# Wit speech API endpoint
API_ENDPOINT = 'https://api.wit.ai/speech'

# Wit.ai api access token
wit_access_token = 'SCVCXGT4LDYO2LTM2PWE5F32VBCIPS6E'


def RecognizeSpeech(AUDIO_FILENAME, num_seconds):
    # record audio of specified length in specified audio file
    record_audio(num_seconds, AUDIO_FILENAME)

    # reading audio
    audio = read_audio(AUDIO_FILENAME)

    # defining headers for HTTP request
    headers = {'authorization': 'Bearer ' + wit_access_token,
               'Content-Type': 'audio/wav'}

    # making an HTTP post request
    resp = requests.post(API_ENDPOINT, headers=headers,
                         data=audio)

    # converting response content to JSON format
    data = json.loads(resp.content)

    # get text from data
    text = data['text']

    # return the text
    return text

def fuzzy(sentence, list_of_items):
    str2match=sentence
    strOptions=list_of_items
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    return highest
def time_(list_of_text):
    now = datetime.datetime.now()
    return (now.hour,now.minute)

def weather_(text):
    reg_ex = re.search('current weather in (.*)', text)
    if reg_ex:
         city = reg_ex.group(1)
         owm = OWM(API_key='ab0d5e80e8dafb2cb81fa9e82431c1fa')
         obs = owm.weather_at_place(city)
         w = obs.get_weather()
         k = w.get_status()
         x = w.get_temperature(unit='celsius')
    return (city, k, x['temp_max'], x['temp_min'])
        
def intent_finder(list_of_text, list_of_items):
    count=0
    l=list()
    for i in list_of_text:
        if i in list_of_items:
            print("the item you're looking for is:",i)
            test="the item you're looking for is"+""+i
            count=1
            break
    if (count==0):
        for i in list_of_text:
            k=fuzzy(i,list_of_items)
            l.append(k)
        res = sorted(l, key = lambda i: i[1], reverse = True)[0][0]
        test="Do you mean"+" "+res
        print("Do you mean : \n",res)
    return test
        


if __name__ == "__main__":  
    try:
    
        text = RecognizeSpeech('myspeech.wav', 5)
    
        print("\nYou said: {}".format(text))
    
        list_of_items=["keys","pen","book","watch","kettle","scissors", "bottle", "wine glass", "cup", "fork", "knife", "bowl","hair dryer","vase","tennis racket"]
        list_of_text=text.split()
        sp = spacy.load('en_core_web_sm')
        all_stopwords = sp.Defaults.stop_words 
        list_of_text_without_sw= [word for word in list_of_text if not word in all_stopwords]
        print(list_of_text_without_sw)
        test=""
    
        if 'time' in list_of_text_without_sw:
            hour_,minutes_=time_(list_of_text_without_sw)
            print('Current time is %d hours %d minutes' % (hour_,minutes_))
            test='Current time is %d hours %d minutes' % (hour_,minutes_)
        elif 'hello' in list_of_text_without_sw or 'hi' in list_of_text_without_sw :
            print("greetings")
            now = datetime.datetime.now()
            day_time = int(now.strftime('%H'))
            print(day_time)
            if day_time < 12:
                print('Hello Sir.Good morning')
                test= 'Hello Sir. Good morning'
            elif 12 <= day_time < 18:
                print('Hello Sir. Good afternoon')
                test= 'Hello Sir. Good afternoon'
            else:
                print('Hello Sir. Good evening')
                test ='Hello Sir. Good evening'
        elif 'weather' in list_of_text_without_sw:
        
            city, k, temp_max, temp_min = weather_(text)
            test = 'It is '+ str(temp_max)+'celsius and'+' '+ k +' '+ 'in'+' '+city 
        else:
            test=intent_finder(list_of_text_without_sw,list_of_items)
        
    except:
        print("Some error in processing")
        test="Sorry!!, I didn't get you"
    
   
        
    
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')   
    en_voice_id = "com.apple.speech.synthesis.voice.Alex"
    engine.setProperty('voice', en_voice_id)
    print(test)
    engine.say(test)
    engine.runAndWait()

Listening...
Finished recording.

You said: what is today's date is my watch
["today's", 'date', 'watch']
the item you're looking for is: watch
the item you're looking for iswatch


In [61]:
elif 'current weather' in command:
     reg_ex = re.search('current weather in (.*)', command)
     if reg_ex:
         city = reg_ex.group(1)
         owm = OWM(API_key='ab0d5e80e8dafb2cb81fa9e82431c1fa')
         obs = owm.weather_at_place(city)
         w = obs.get_weather()
         k = w.get_status()
         x = w.get_temperature(unit='celsius')
         sofiaResponse('Current weather in %s is %s. The maximum temperature is %0.2f and the minimum temperature is %0.2f degree celcius' % (city, k, x['temp_max'], x['temp_min']))

SyntaxError: invalid syntax (<ipython-input-61-c7b323b09fc5>, line 1)

In [10]:
count=0
for i in list_of_text:
    if i in list_of_items:
        print("the item you're looking for is:",i)
        count=1
        break
if (count==0):
    for i in list_of_text:
        k=fuzzy(i)
        if k[1] >=80:
            print("Do you mean:"+" "+k[0])
            break

In [44]:
print(l[0][0])
max=0
for i in l:
    print(i)
    if max < i[1]:
        max=i[1]
    

wine glass
('wine glass', 45)
('watch', 40)
('pen', 60)
('tennis racket', 75)
('watch', 40)
('scissors', 90)
('keys', 45)
('tennis racket', 90)


In [55]:

res = sorted(l, key = lambda i: i[1], reverse = True)[0][0]
print("Do you mean : \n",res)

[('wine glass', 45), ('watch', 40), ('pen', 60), ('tennis racket', 75), ('watch', 40), ('scissors', 90), ('keys', 45), ('tennis racket', 90)]
Day with maximum score is : 
 scissors


In [71]:
import pyttsx3
engine = pyttsx3.init()
voices = engine.getProperty('voices')   
en_voice_id = "com.apple.speech.synthesis.voice.Alex"
engine.setProperty('voice', en_voice_id)
engine.say(test)
engine.runAndWait()

In [ ]:
import requests
import json
from fuzzywuzzy import process
import pyaudio
import wave
import pyttsx3
import datetime

def record_audio(RECORD_SECONDS, WAVE_OUTPUT_FILENAME):
    # --------- SETTING PARAMS FOR OUR AUDIO FILE ------------#
    FORMAT = pyaudio.paInt16  # format of wave
    CHANNELS = 2  # no. of audio channels
    RATE = 44100  # frame rate
    CHUNK = 1024  # frames per audio sample
    # --------------------------------------------------------#

    # creating PyAudio object
    audio = pyaudio.PyAudio()

    # open a new stream for microphone
    # It creates a PortAudio Stream Wrapper class object
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    # ----------------- start of recording -------------------#
    print("Listening...")

    # list to save all audio frames
    frames = []

    for i in range(int(RATE / CHUNK * RECORD_SECONDS)):
        # read audio stream from microphone
        data = stream.read(CHUNK)
        # append audio data to frames list
        frames.append(data)

    # ------------------ end of recording --------------------#
    print("Finished recording.")

    stream.stop_stream()  # stop the stream object
    stream.close()  # close the stream object
    audio.terminate()  # terminate PortAudio

    # ------------------ saving audio ------------------------#

    # create wave file object
    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')

    # settings for wave file object
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))

    # closing the wave file object
    waveFile.close()


def read_audio(WAVE_FILENAME):
    # function to read audio(wav) file
    with open(WAVE_FILENAME, 'rb') as f:
        audio = f.read()
    return audio

# Wit speech API endpoint
API_ENDPOINT = 'https://api.wit.ai/speech'

# Wit.ai api access token
wit_access_token = 'SCVCXGT4LDYO2LTM2PWE5F32VBCIPS6E'


def RecognizeSpeech(AUDIO_FILENAME, num_seconds):
    # record audio of specified length in specified audio file
    record_audio(num_seconds, AUDIO_FILENAME)

    # reading audio
    audio = read_audio(AUDIO_FILENAME)

    # defining headers for HTTP request
    headers = {'authorization': 'Bearer ' + wit_access_token,
               'Content-Type': 'audio/wav'}

    # making an HTTP post request
    resp = requests.post(API_ENDPOINT, headers=headers,
                         data=audio)

    # converting response content to JSON format
    data = json.loads(resp.content)

    # get text from data
    text = data['text']

    # return the text
    return text

def fuzzy(sentence):
    str2match=sentence
    strOptions=list_of_items
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    return highest
def time_(list_of_text):
     if 'time' in list_of_text:
           now = datetime.datetime.now()
           return (now.hour,now.minute)


if __name__ == "__main__":
    text = RecognizeSpeech('myspeech.wav', 5)
    
    print("\nYou said: {}".format(text))
    
    list_of_items=["keys","pen","book","watch","kettle","scissors", "bottle", "wine glass", "cup", "fork", "knife", "bowl","hair drier","vase"]
    list_of_text=text.split()
    
    count=0
    l=list()
    test=""
    for i in list_of_text:
        if i in list_of_items:
           print("the item you're looking for is:",i)
           test="the item you're looking for is"+i
           count=1
           break
    if (count==0):
        for i in list_of_text:
            k=fuzzy(i)
            l.append(k)
        res = sorted(l, key = lambda i: i[1], reverse = True)[0][0]
        test="Do you mean"+res
        print("Do you mean : \n",res)
        
    elif
    
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')   
    en_voice_id = "com.apple.speech.synthesis.voice.Alex"
    engine.setProperty('voice', en_voice_id)
    engine.say(test)
    engine.runAndWait()